### Import Libraries

In [ ]:
!pip install pysteps
!pip install kagglehub
from google.colab import drive
drive.mount('/content/drive/')
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, ConcatDataset, Dataset, TensorDataset
from pysteps.utils import spectral
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import kagglehub

### Move to GPU

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Parameters

In [ ]:
# To be modified or enhanced
batch_size = 32
num_epochs = 2
learning_rate = 0.001
rapsd_dim = 16*3
n_classes = 2

### Load Datasets

Datadream

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

synthetic_path = '/content/drive/MyDrive/SharedFolder/Project18/synthetic'
real_path = '/content/drive/MyDrive/SharedFolder/Project18/real'

# Load datasets with ImageFolder
synthetic_datadream = datasets.ImageFolder(root=synthetic_path, transform=transform)
real_datadream = datasets.ImageFolder(root=real_path, transform=transform)

Cifake

In [ ]:
# Download CIFAKE dataset (CIFAR-10 subset and synthetic versions of each image)
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")
print("Path to dataset files:", path)

cifake_images_path_test = path + "/test"
cifake_images_path_train = path + "/train"

# Load datasets CIFAKE train and test
cifake_dataset_train = datasets.ImageFolder(root=cifake_images_path_train, transform=transform)
cifake_dataset_test = datasets.ImageFolder(root=cifake_images_path_test, transform=transform)

### Functions

In [ ]:
# Function to prepare image for fft
def prepare_image_for_fft(image):

    # Convert to numpy array
    image_np = image.cpu().numpy()
    return image_np

In [ ]:
def compute_rapsd_subset(subset):
    """
    Computes RAPSD for a subset of images.

    Parameters:
    - subset: List or dataset of (image, label) pairs.

    Returns:
    - List of merged RAPSD features for each image.
    - List of frequency arrays for each image.
    """
    rapsd_subset = []
    frequencies_subset = []

    # For all images in subset
    for img, _ in subset:
        rapsd_features = []  # To store per-channel RAPSD for one image
        frequencies_list = []  # To store per-channel frequencies for one image

        # Iterate over each of the three channels (RGB)
        for i in range(img.shape[0]):  # Assuming the image is 3xHxW (3 channels)
            # Prepare the image
            prepared_image = prepare_image_for_fft(img[i])

            # Compute RAPSD
            rapsd, frequencies = spectral.rapsd(prepared_image, fft_method=np.fft, return_freq=True)
            rapsd_features.append(rapsd)
            frequencies_list.append(frequencies)

        # Merge RAPSD features for all channels of one image
        merged_rapsd = np.concatenate(rapsd_features, axis=0)  # Combine RAPSD features for all channels
        merged_frequencies = np.array(frequencies_list[0])  # Use frequencies from the first channel (identical)

        # Append per-image results
        rapsd_subset.append(merged_rapsd)
        frequencies_subset.append(merged_frequencies)

    return rapsd_subset, frequencies_subset

In [ ]:
def accuracy(target, pred):
  target = target.detach().cpu().numpy()
  pred = pred.detach().cpu().numpy()
  return metrics.accuracy_score(target, pred)

In [ ]:
def compute_confusion_matrix(target, pred, normalize=None):
    return metrics.confusion_matrix(
        target.detach().cpu().numpy(),
        pred.detach().cpu().numpy(),
        normalize=normalize
    )

### Image Preprocessing

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

### Statistical tests on RAPSD

Student's t

Cohen's d

### Models